In [701]:
from tqdm.auto import tqdm
import numpy as np
import json, glob, os
import pandas as pd
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
import math

In [702]:
def read_(path):
    with open(path, "r") as f:
        return f.readlines()

In [715]:
lex = read_('data/english-hindi-dictionary.txt')
eng = read_('data/english.txt')
hindi = read_('data/hindi.txt')
out = read_('data/BingLiu.csv')

In [716]:
lex = { i.split('|||')[0].strip():i.split('|||')[-1].strip() for i in lex}
# lex

In [717]:
# # saving for glove 
# a = ''
# for i in eng:
#     a+=i.strip().lower() + ' '
    
# with open('GloVe/eng.txt', 'w') as f:
#     f.write(a)

## L1 

In [718]:
# en_word1, hindi_word1, polarity

In [719]:
# a = {i.split('\t')[0] : i.split('\t')[1] for i in out}    
out = [[j.strip() for j in i.split('\t')] for i in out]

In [720]:
out[:4], len(out)

([['a+', 'positive'],
  ['abound', 'positive'],
  ['abounds', 'positive'],
  ['abundance', 'positive']],
 6789)

In [765]:
L1 = ''
for i in tqdm(out):
    try: L1 += f"{i[0]},{lex[i[0]]},{i[-1]}\n"
    except: pass #L1 += f"{i[0]},{None},{i[-1]}\n"


In [766]:
print(len(L1.split('\n')))

5014


## Embeddings

In [767]:
eng_ = [[j for j in i.split()]for i in eng]
hindi_ = [[j for j in i.split()]for i in hindi]

In [768]:
# w2v_eng = Word2Vec(eng_, size=100, window=5, min_count=1, workers=48, iter=1000)
# w2v_hindi = Word2Vec(hindi_, size=100, window=5, min_count=1, workers=48, iter=1000)
# w2v_eng.save("w2v_eng.model")
# w2v_hindi.save("w2v_hindi.model")
w2v_eng = Word2Vec.load("w2v_eng.model")
w2v_hindi = Word2Vec.load("w2v_hindi.model")
# model.most_similar('used')[:5]

In [769]:
def m_(d):
    return [float(i) for i in d]
def glove(path):
    return { i.split()[0]:i.split()[1:] for i in read_(path)}

In [770]:
glove_eng = glove('data/eng/vectors.txt')
glove_hindi = glove('data/hindi/vectors.txt')
len(glove_eng), len(glove_hindi)

(8074, 10354)

## Words matching

In [771]:
# English using Word2Vec
closest = {i:[] for i in L1.split('\n')}
n = 5
a = 0
for i in tqdm(L1.split('\n')):
    try: 
        closest[i].append([i[0] for i in w2v_eng.most_similar(i.split(',')[0])][:n])
        a+=1
    except: closest[i].append([None])
        
print(f"Number of words matched:- {a}")
print(closest['acclaimed,acclaimed,positive'])

#Hindi using Word2Vec
# closest = {}
n = 5
a = 0
for i in tqdm(L1.split('\n')):
    try: 
        closest[i].append([i[0] for i in w2v_hindi.most_similar(i.split(',')[1])][:n])
        a+=1
    except: closest[i].append([None])
print(f"Number of words matched:- {a}")
print(closest['accomplishment,उपलब्धि,positive'])

<ipython-input-771-6a742fd91459>:7: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  closest[i].append([i[0] for i in w2v_eng.most_similar(i.split(',')[0])][:n])



Number of words matched:- 826
[['multi-touch', 'gestures', 'scrolling', "Apple's", 'recommend!!']]


<ipython-input-771-6a742fd91459>:20: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  closest[i].append([i[0] for i in w2v_hindi.most_similar(i.split(',')[1])][:n])



Number of words matched:- 662
[[None], ['कटारिया', 'हरिद्वार', 'शरद', 'बनारस', 'तात्पर्य']]


In [772]:
closest

{'abound,abound,positive': [[None], [None]],
 'abounds,abounds,positive': [[None], [None]],
 'abundance,abundance,positive': [[None], [None]],
 'abundant,हुस्न,positive': [[None], [None]],
 'accessable,accessable,positive': [[None], [None]],
 'accessible,सुलभ,positive': [[None], [None]],
 'acclaim,acclaim,positive': [[None], [None]],
 'acclaimed,acclaimed,positive': [['multi-touch',
   'gestures',
   'scrolling',
   "Apple's",
   'recommend!!'],
  [None]],
 'accolade,accolade,positive': [[None], [None]],
 'accolades,accolades,positive': [['suspected',
   'justifies',
   'receives',
   'standard.',
   'fries'],
  [None]],
 'accomplish,पूरा,positive': [[None],
  ['कॉलिंग', 'एक्सटर्नल', 'सातों', 'बयां', 'पूरे']],
 'accomplished,पूरा हुआ,positive': [['Product',
   'Key.',
   'upgrades',
   'resolved',
   'pin'],
  [None]],
 'accomplishment,उपलब्धि,positive': [[None],
  ['कटारिया', 'हरिद्वार', 'शरद', 'बनारस', 'तात्पर्य']],
 'accomplishments,उपलब्धियों,positive': [[None], [None]],
 'accurate

In [787]:
d = 0
m = ''
for k,v in closest.items():
    if v[0][0] != None and v[1][0] !=None:
        print(v[0],v[1])
        for j in v[0]:
            try: print(j.strip(),lex[j])
            except: print(j)
#         m += f"{i[0]},{lex[i[0]]},{i[-1]}\n"
        d += 1
        break
d

['technologically', 'tasks', 'features.', 'edit.', 'point.'] ['इटरेक्टिव', 'मेगाहर्ट्ज', 'Qualcomm', 'एक्सर्वस्ट', 'पुनः']
technologically technologically
tasks अधूरे कार्य
features.
edit.
point.


1

In [731]:
def cosine_similarity(v1,v2):
    "compute cosine similarity of v1 to v2: (v1 dot v2)/{||v1||*||v2||)"
    sumxx, sumxy, sumyy = 0, 0, 0
    for i in range(len(v1)):
        x = v1[i]; y = v2[i]
        sumxx += x*x
        sumyy += y*y
        sumxy += x*y
    return sumxy/math.sqrt(sumxx*sumyy)

# v1,v2 = [3, 45, 7, 2], [2, 54, 13, 15]
# cosine_similarity(v1,v2)

In [732]:
#English using Glove
closest = {}
n = 5
a = 0
for i in tqdm(L1.split('\n')):
    word = i.split(',')[0]
    if word in glove_eng.keys():
        dummy = {}
        for j in glove_eng.keys():
            score = cosine_similarity([float(i) for i in glove_eng[j][-100:]],[float(i) for i in glove_eng[word][-100:]])
            dummy[score] = j
        closest[i] = [i[-1] for i in sorted(dummy.items(),reverse=True)][1:6] 
        a+=1
#         break
    else: closest[i] = None
f"Number of words matched:- {a}"

'Number of words matched:- 867'

In [733]:
closest['acclaimed,acclaimed,positive']

['this!!', 'would', 'easly', 'droped', 'viruses!']

In [700]:
#Hindi using Glove
closest = {}
n = 5
a = 0
for i in tqdm(L1.split('\n')):
    word = i.split(',')[0]
    if word in glove_hindi.keys():
        dummy = {}
        for j in glove_hindi.keys():
            score = cosine_similarity([float(i) for i in glove_hindi[j][-100:]],[float(i) for i in glove_hindi[word][-100:]])
            dummy[score] = j
        closest[i] = [i[-1] for i in sorted(dummy.items(),reverse=True)][1:6] 
        a+=1
#         break
    else: closest[i] = None
f"Number of words matched:- {a}"

ValueError: Expected 2D array, got 1D array instead:
array=[-0.952942  0.661924 -0.408228  0.274482  1.498098  1.317056 -0.051193
  0.536412  0.268136  0.597217  1.251104 -0.409273 -0.764494  0.937964
  0.594754  1.250672 -1.044761  1.433465 -0.502353 -0.116605 -1.538433
  0.555333  1.230167 -0.341261  0.016535  1.079324 -0.151853  0.096634
  0.834354  0.524879 -0.735272  0.020952 -0.021831  1.1425    1.561613
  0.036755  0.641463  0.355424  1.20845   0.647668 -0.852669 -0.498579
 -0.017966 -0.227793 -1.393405 -0.330712  0.423414  0.552423 -0.101748
 -0.063826  0.503883  0.524101  0.998811 -0.785256 -0.192301 -0.068441
 -0.505361  0.760461 -0.904783 -1.664649  0.43836   0.613362  1.09256
 -1.070104  0.13658   0.250102  1.150277  0.340985 -0.372592 -0.276505
 -0.29722  -0.708953 -0.007783  0.721822 -2.019738  1.332947 -1.518888
 -0.637268 -0.165223  0.210054  0.486783 -0.084931 -0.501394 -0.561859
  0.101731  0.185223 -0.021546 -0.977347  0.528597 -0.10906   0.380566
  0.080269  0.82254

In [792]:
closest['acclaimed,acclaimed,positive']

[['multi-touch', 'gestures', 'scrolling', "Apple's", 'recommend!!'], [None]]